## Code for Paper Diabetic Retinopathy Diagnosis

Importing the libraries

In [ ]:
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

### Building the Model

In [ ]:
#Change Model name for different models
# ResNet101V2 EfficientNetB3 DenseNet169

baseModel = tf.keras.applications.Xception(weights="imagenet", include_top=False,input_tensor=tf.keras.layers.Input(shape=(299, 299, 3)))

headModel = baseModel.output
headModel = tf.keras.layers.GlobalAveragePooling2D()(headModel)
headModel = tf.keras.layers.Dense(256, activation="relu")(headModel)
headModel = tf.keras.layers.Dropout(0.4)(headModel)


In [ ]:
#Choose between 2 and 8 for binary and 8-ary classification models
headModel = tf.keras.layers.Dense(2, activation="softmax")(headModel)

model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)

#Choose between binary_crossentropy and categorical_crossentropy

model.compile(loss="categorical_crossentropy", optimizer="nadam",metrics=["accuracy"])

### Dataset Selection and Augmentation

In [ ]:
#Dataset Augmentation

trainAug = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,shear_range=10.0,zoom_range=[0.90,1.10],channel_shift_range=10)
valAug = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,shear_range=10.0,zoom_range=[0.90,1.10],channel_shift_range=10)
testAug = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False)

In [ ]:
#Choose Dataset for training

class_select = "binary_data" # "5ary_data"
train_select = "kaggle"
test_select = "messidor"  #"idrid" "kaggle/test"

trainGen = trainAug.flow_from_directory(
	"data/"+class_select+"/"+train_select+"/train",
	class_mode="categorical",
	target_size=(299, 299),
	color_mode="rgb",
	shuffle=True,
	batch_size=16)


# initialize the validation generator
valGen = valAug.flow_from_directory(
	"data/"+class_select+"/"+train_select+"/val",
	class_mode="categorical",
	target_size=(299, 299),
	color_mode="rgb",
	shuffle=False,
	batch_size=16)


# initialize the testing generator
testGen = testAug.flow_from_directory(
	"data/"+class_select+"/"+test_select+"/",
	class_mode="categorical",
	target_size=(299, 299),
	color_mode="rgb",
	shuffle=False,
	batch_size=16)

### Model checkpoint and callbacks

In [ ]:
mod = "xception_models/" # "efficientnet_models/"  "insres_models/"
mod1 = "xception" # "B3"  "insres"
classo= "5ary" # "binary"

checkpoint_filepath = "models/"+mod+mod1+"_"+classo

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True)

### Model training 

In [ ]:
#Change the steps according to the train/val generator values

history = model.fit_generator(
	trainGen,
	steps_per_epoch= 1453//16 ,
	validation_data=valGen,
	validation_steps= 177//16,
	epochs=50,
    callbacks=[model_checkpoint_callback])

### Model performance testing

In [ ]:
modtest = tf.keras.models.load_model(checkpoint_filepath)

In [ ]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = modtest.predict_generator(testGen)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))